In [13]:
from sentence_transformers import SentenceTransformer


model_name_or_path = '../../cache/officials/bge-large-zh-v1.5'

model = SentenceTransformer(model_name_or_path, device='cuda:5')

print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [8]:
from sentence_transformers import SentenceTransformer


model_name_or_path = '../../cache/officials/all-MiniLM-L6-v2'

model = SentenceTransformer(model_name_or_path, device='cuda:5')

print(model)

No sentence-transformers model found with name ../../cache/officials/all-MiniLM-L6-v2. Creating a new one with mean pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [1]:
from datasets import load_dataset


ds = load_dataset("sentence-transformers/stsb")

In [3]:
ds['train'][0]

{'sentence1': 'A plane is taking off.',
 'sentence2': 'An air plane is taking off.',
 'score': 1.0}

In [4]:
import json
from datasets import Dataset

def json2triplet(filepath:str):
    
    l_anchor = []
    l_positive = []
    l_negative = []
    
    with open(filepath) as file:
        for line in file:
            tmp_dict = json.loads(line)
            anchor = tmp_dict['query']
            for pos in tmp_dict['pos']:
                for neg in tmp_dict['neg']:
                    l_anchor.append(anchor)
                    l_positive.append(pos)
                    l_negative.append(neg)
                    
    data = {
        "anchor": l_anchor,
        "positive": l_positive,
        "negative": l_negative,
    }
    return data

def json2pairswithscore(filepath:str):
    
    l_sentences_1 = []
    l_sentences_2 = []
    l_score = []
    
    with open(filepath) as file:
        for line in file:
            tmp_dict = json.loads(line)
            anchor = tmp_dict['query']
            for pos in tmp_dict['pos']:
                l_sentences_1.append(anchor)
                l_sentences_2.append(pos)
                l_score.append(1)
            for neg in tmp_dict['neg']:
                l_sentences_1.append(anchor)
                l_sentences_2.append(neg)
                l_score.append(-1)
                
    data = {
        "sentence1": l_sentences_1,
        "sentence2": l_sentences_2,
        "score": l_score,
    }
    return data
                

            

file = '/data02/hyzhang10/pengxia2/PrivateCollection/Embedding/data/valid_embedding_and_reranker_data.json'
# pairswithscore = json2pairswithscore(file)
# train_dataset = Dataset.from_dict(pairswithscore)

triplets = json2triplet(file)
train_dataset = Dataset.from_dict(triplets)
# len(triplets['anchor'])

In [5]:
train_dataset

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 10868
})

In [12]:
train_dataset[0]

{'sentence1': '矢板明夫是如何评价1951年和平解放西藏协议的执行情况？',
 'sentence2': '。\n退役上校吁成台湾特区 矢板明夫：是“自毁长城”\n \n \n \n \n \n \n \n \n \n \n \n 人气: 28\n \n \n 【字号】\n 大\xa0\n 中\xa0\n 小\n \n \n \n 更新: 2022-12-27 8:29 PM \n \xa0\xa0 标签: tags:\n 矢板明夫, 黄征辉, 统一, 统派 \n \n \r\n【大纪元2022年12月27日讯】（大纪元记者吴旻洲台湾台北报导）中共持续对台文攻武吓，有部分统派人士认为，台湾应该与中共谈判，以保留台湾民主自由为前提与中共统一。对此，日本产经新闻台北支局长矢板明夫表示，此时与中共谈判等于是签“城下之盟”，将导致以美国为首的国家无法介入、协防台湾，无疑是“自毁长城”。\n\r\n海军军舰舰长黄征辉近日在节目访谈中表示，他认为台湾应该早日和中国统一，成为中华人民共和国的台湾特区。条件是要保持现在台湾民主自由的社会制度、保留军队。如果能够实现的话，台湾将不会有被对方侵略的危险，两岸人民可以安居乐业、共同发展经济。\n\r\n黄征辉认为，台湾需要付出的，只是“更改一个名字”而已，中共得到了面子，台湾得到了里子，何乐而不为？如果将来中共反悔、非要改变台湾的社会制度的话，台湾有军队可以自保。\n\r\n对此，矢板明夫表示，黄征辉的主张并非少数意见，在台湾统派当中有一定市场。他们认为，民生和经济最重要，“何必为了一个名字闹得兵戎相见呢？”但作为一个长期观察中国共产党的媒体人，他认为这样的想法过于一厢情愿了，根本不现实。\n\r\n他表示，今天的台海局势，是对岸的军队随时可能攻打台湾，而台湾的军队并没有任何攻打对岸的计划。台湾如果为了消除对自己的军事威胁，主动去找中共谈判，等于是签“城下之盟”。\n\r\n他说，中共这么多年没有攻打台湾，是因为忌讳以美国为首的国际社会的介入。以现在中共的实力，还不敢与美国抗衡。但城下之盟一旦签下去，台湾问题将彻底变成中国的“内政问题”，国际社会就不能介入，不能再卖武器给台湾，也不能协防台湾。对台湾而言，无疑相当于“自毁长城”。\n\r\n矢板明夫说，和独裁国家签合约，对方基本上不会遵守，他以1951年“和平解放西藏协议”为